In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/nfl_plays.csv')

/Users/Philip/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (15,16,17,22,27,33,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
df.isnull().sum()

GameId                            0
GameDate                          0
Quarter                           0
Minute                            0
Second                            0
OffenseTeam                       0
DefenseTeam                       0
Down                              0
ToGo                              0
YardLine                          0
SeriesFirstDown                   0
NextScore                         0
Description                       0
TeamWin                           0
SeasonYear                        0
Yards                             0
Formation                         0
PlayType                          0
IsRush                            0
IsPass                            0
IsIncomplete                      0
IsTouchdown                       0
PassType                          0
IsSack                            0
IsChallenge                       0
IsChallengeReversed               0
IsMeasurement                     0
IsInterception              

In [4]:
# remove unnamed columns. Most are empty seperators
df = df.drop([u'Unnamed: 10', u'Unnamed: 12', u'Unnamed: 16', u'Unnamed: 17', 
              u'Unnamed: 45','Challenger'], axis=1)

In [25]:
# Check why columns have null values
df[df[u'OffenseTeam'].isnull()].head(10)
df[df[u'Description'].isnull()].head(10)
df[df[u'SeasonYear'].isnull()].head(10)
df[df['Yards'].isnull()].head(10)
df[df['Formation'].isnull()].head(10)
df[df['PlayType'].isnull()].head(10)
df[df['IsPass'].isnull()].head(10)
df[df['IsIncomplete'].isnull()].head(10)
df[df['IsTouchdown'].isnull()].head(10)
df[df['PassType'].isnull()].head(10)
df[df['IsSack'].isnull()].head(10)
df[df['IsTwoPointConversion'].isnull()].head(10)
df[df['YardLineFixed'].isnull()].head(10)
df[df['IsNoPlay'].isnull()].head(10)
df[df['PenaltyYards'].isnull()].head(10)
df[df['OffenseTeam'].isnull()].head(10)
df[df['Formation'].isnull()].head(10)
df[df['RushDirection'].isnull()].head(10)
df[df['PenaltyTeam'].isnull()].head(10)
df[df['PenaltyType'].isnull()].head(10)

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards


In [6]:
# turn gamedate into datetimes
df['GameDate'] = df['GameDate'].apply(pd.to_datetime)

In [7]:
# replace null values of 'SeasonYear' with 2013
mask = df['SeasonYear'].isnull()
df.loc[mask, 'SeasonYear'] = 2013.0

In [8]:
# replace null values of 'Yards' with 0.0
mask = df['Yards'].isnull()
df.loc[mask, 'Yards'] = 0.0

In [9]:
# replace null values of 'PlayType' with 'RUSH', they are wildcat plays
mask = df['PlayType'].isnull()
df.loc[mask, 'PlayType'] = 'RUSH'

In [10]:
# replace null values of 'IsPass' with 0
mask = df['IsPass'].isnull()
df.loc[mask, 'IsPass'] = 0

In [11]:
# replace null values of 'IsTouchdown' with 0
mask = df['IsTouchdown'].isnull()
df.loc[mask, 'IsTouchdown'] = 0

In [12]:
# replace null values of 'PassType' with 0, also kicks were being missclassified as PASS
mask = df['PlayType'] != 'PASS'
df.loc[mask, 'PassType'] = 0

mask = (df['PlayType'] == 'PASS') & df['Description'].str.contains('KICKS')
df.loc[mask, 'PassType'] = 0
df.loc[mask, 'PlayType'] = 'PUNT'

mask = (df['PlayType'] == 'PASS') & df['Description'].str.contains('FIELD GOAL')
df.loc[mask, 'PassType'] = 0
df.loc[mask, 'PlayType'] = 'FIELD GOAL'

mask = (df['PlayType'] == 'PASS') & df['Description'].str.contains('EXTRA POINT')
df.loc[mask, 'PassType'] = 0
df.loc[mask, 'PlayType'] = 'EXTRA POINT'

mask = (df['PlayType'] == 'PASS') & df['Description'].str.contains('KNEELS')
df.loc[mask, 'PassType'] = 0
df.loc[mask, 'PlayType'] = 'QB KNEEL'

mask = (df['PlayType'] == 'PASS') & df['Description'].str.contains('END GAME')
df.loc[mask, 'PassType'] = 0
df.loc[mask, 'PlayType'] = 'NO PLAY'

In [13]:
# replace null values of 'IsIncomplete' with 0
mask = df['IsIncomplete'].isnull()
df[mask] = df[mask].fillna(0)

In [14]:
# replace all of the null values in one faulty row with 0's
mask = df['IsSack'].isnull()
df[mask] = df[mask].fillna(0)

In [15]:
# replace null values of 'IsMeasurement' with 0
df['IsMeasurement'] = df['IsMeasurement'].fillna(0)

In [16]:
mask = df['YardLineFixed'].isnull()
df.loc[mask, 'YardLineFixed'] = 100 - df[mask]['YardLine'] 

In [17]:
# replace null values of 'IsNoPlay' with 0
df['IsNoPlay'] = df['IsNoPlay'].fillna(0)

In [18]:
# replace null values of 'PenaltyYards' with 0
df['PenaltyYards'] = df['PenaltyYards'].fillna(0)

In [19]:
mask = (df['OffenseTeam'].isnull()) & df['Description'].str.contains('END QUARTER')
df.loc[mask, 'OffenseTeam'] = 0
df.loc[mask, 'PlayType'] = 'END QUARTER'

mask = (df['OffenseTeam'].isnull()) & df['Description'].str.contains('TIMEOUT')
df.loc[mask, 'OffenseTeam'] = 0
df.loc[mask, 'PlayType'] = 'TIMEOUT'

mask = (df['OffenseTeam'].isnull()) & df['Description'].str.contains('END GAME')
df.loc[mask, 'OffenseTeam'] = 0
df.loc[mask, 'PlayType'] = 'END GAME'

mask = (df['OffenseTeam'].isnull()) & df['Description'].str.contains('TWO-MINUTE WARNING')
df.loc[mask, 'OffenseTeam'] = 0
df.loc[mask, 'PlayType'] = 'TWO-MINUTE WARNING'

mask = (df['OffenseTeam'].isnull()) & df['Description'].str.contains('END OF')
df.loc[mask, 'OffenseTeam'] = 0
df.loc[mask, 'PlayType'] = 'END OF QUARTER/HALF'

mask = (df['OffenseTeam'].isnull()) & df['Description'].str.contains('PASS')
df.loc[mask, 'OffenseTeam'] = 0
df.loc[mask, 'PlayType'] = 'END OF QUARTER/HALF'

# drop the rest and include description
df.dropna(subset=['OffenseTeam', 'Description'], inplace = True)

In [20]:
# replace null values of 'Formation' with 0
df['Formation'] = df['Formation'].fillna(0)

In [21]:
mask = (df['RushDirection'].isnull()) & (df['PlayType'] == 'RUSH') & (df['Description'].str.contains('CENTER'))
df.loc[mask, 'RushDirection'] = 'CENTER'

mask = (df['RushDirection'].isnull()) & (df['PlayType'] == 'RUSH') & (df['Description'].str.contains('LEFT TACKLE'))
df.loc[mask, 'RushDirection'] = 'LEFT TACKLE'

mask = (df['RushDirection'].isnull()) & (df['PlayType'] == 'RUSH') & (df['Description'].str.contains('RIGHT GUARD'))
df.loc[mask, 'RushDirection'] = 'RIGHT GUARD'

mask = (df['RushDirection'].isnull()) & (df['PlayType'] == 'RUSH') & (df['Description'].str.contains('LEFT GUARD'))
df.loc[mask, 'RushDirection'] = 'LEFT GUARD'

mask = (df['RushDirection'].isnull()) & (df['PlayType'] == 'RUSH') & (df['Description'].str.contains('RIGHT TACKLE'))
df.loc[mask, 'RushDirection'] = 'RIGHT TACKLE'

mask = (df['RushDirection'].isnull()) & (df['PlayType'] == 'RUSH') & (df['Description'].str.contains('RIGHT END'))
df.loc[mask, 'RushDirection'] = 'RIGHT END'

mask = (df['RushDirection'].isnull()) & (df['PlayType'] == 'RUSH') & (df['Description'].str.contains('LEFT END'))
df.loc[mask, 'RushDirection'] = 'LEFT END'

# fill the rest with 0
mask = df['RushDirection'].isnull()
df[mask] = df[mask].fillna(0)

In [22]:
df['PenaltyTeam'] = df['PenaltyTeam'].fillna(0)

In [23]:
df['PenaltyType'] = df['PenaltyType'].fillna(0)